<a href="https://colab.research.google.com/github/namwootree/Portfolio/blob/main/Alphaco_(Deep_Learning_Boot_Camp)/Long-Term%20Program/Text_Summarization/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
from pprint import pprint

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

In [3]:
train_json_path = '/content/drive/MyDrive/장기 프로젝트/문서 요약/235813_AI 기반 회의 녹취록 요약 경진대회_data/train.json'

with open(train_json_path) as train_file:
  TRAIN_DATA = json.load(train_file)

# JSON

In [6]:
pprint(TRAIN_DATA[1])

{'context': {'AGENDA_1': {'0': '의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 '
                               '정례회 제1차 본회의 개의를 선포합니다.',
                          '1': '먼저 의사팀장으로부터 의회 관련 사항에 대한 보고가 있겠습니다. 의사팀장은 보고하여 '
                               '주시기 바랍니다.',
                          '10': '완주 혁신도시 출판복합 클러스터 조성에 총력 기울여야.',
                          '11': '존경하는 완주군민 여러분! 최등원 의장님과 선배·동료 의원 여러분! 박성일 '
                                '군수님을 비롯한 공무원 및 관계자 여러분! 더불어민주당 삼례·이서 출신 윤수봉 '
                                '의원입니다. 전북 혁신도시에 출판산업 복합 클러스터를 조성하여 호남권역 및 국내 '
                                '출판산업의 메카로 육성해야 한다는 도민의 염원과 당위성이 고조되고 있습니다. 완주 '
                                '혁신도시의 출판문화산업진흥원이 주축이 되어 해당 부처의 협의를 이끌어내고 '
                                '전라북도와 완주군이 적극 힘을 보탠 결과입니다. 의회 차원에서도 지난 2월 '
                                '전라북도 시군의장단협의회를 통해 본 사안을 촉구하는 건의안을 제시한 바 있습니다. '
                                '전북 혁신도시의 출판복합 클러스터 조성은 두 가지 측면에서 반드시 성사되어야 '
                                '합니다.',
    

# Context DataFrame

In [12]:
train_1 = pd.DataFrame(columns=['uid', 'title', 'region', 'agenda', 'context', 'summary'])

uid = 1000

for data in TRAIN_DATA:

  for agenda in data['context'].keys():
    
    context = ''

    for line in data['context'][agenda]:

      context += data['context'][agenda][line]
      context += ''

    train_1.loc[uid, 'uid'] = uid
    train_1.loc[uid, 'title'] = data['title']
    train_1.loc[uid, 'region'] = data['region']
    train_1.loc[uid, 'agenda'] = agenda
    train_1.loc[uid, 'context'] = context[:-1]
    train_1.loc[uid, 'summary'] = data['label'][agenda]['summary']
    
    uid += 1

train_1['total'] = train_1.title + ' ' + train_1.region + ' ' + train_1.context

In [13]:
list_title = list(train_1['title'].unique())
list_id = [2000 + int(i) for i in range(len(list_title))]

for id, title in zip(list_id, list_title):

  cond = (train_1['title']==title)
  train_1.loc[cond, 'id'] = id

train_1 = train_1.astype({'id':'int'})
train_1 = train_1[['id', 'title', 'region', 'agenda', 'context', 'summary', 'total']]

In [14]:
train_1.head()

,id,title,region,agenda,context,summary,total
1000,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_1,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의석을 정돈하여 주시기 ...
1001,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_2,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의사팀장 수고하셨습니다....
1002,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_3,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제2항 ...
1003,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_4,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제3항 ...
1004,2001,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,AGENDA_1,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록 완주 의석을 정돈...


# Evidence DataFrame

In [75]:
train_2 = pd.DataFrame(columns=['uid', 'title', 'region', 'agenda', 'evidence', 'summary'])

uid = 1000

for data in TRAIN_DATA:

  for agenda in data['label'].keys():

    evidence = ''

    for num in data['label'][agenda]['evidence']:

      evidence += data['context'][agenda][num]
      evidence += ''
    
    train_2.loc[uid, 'uid'] = uid
    train_2.loc[uid, 'title'] = data['title']
    train_2.loc[uid, 'region'] = data['region']
    train_2.loc[uid, 'agenda'] = agenda
    train_2.loc[uid, 'evidence'] = evidence[:-1]
    train_2.loc[uid, 'summary'] = data['label'][agenda]['summary']

    uid += 1   

In [76]:
train_2.head()

,uid,title,region,agenda,evidence,summary
1000,1000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_1,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1001,1001,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_2,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
1002,1002,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_3,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
1003,1003,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_4,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
1004,1004,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,AGENDA_1,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.


In [77]:
list_title = list(train_2['title'].unique())
list_id = [2000 + int(i) for i in range(len(list_title))]

for id, title in zip(list_id, list_title):

  cond = (train_1['title']==title)
  train_2.loc[cond, 'id'] = id

train_2 = train_2.astype({'id':'int'})
train_2 = train_2[['id', 'title', 'region', 'agenda', 'evidence', 'summary',]]

In [78]:
train_2.head()

,id,title,region,agenda,evidence,summary
1000,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_1,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.
1001,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_2,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.
1002,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_3,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.
1003,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_4,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...
1004,2001,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,AGENDA_1,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.


# Merge

In [80]:
train_1.shape, train_2.shape

((2994, 7), (2994, 6))

In [81]:
train = pd.merge(train_1, train_2, how='outer', on=['id', 'title', 'region', 'agenda', 'summary'])

In [82]:
train.head()

,id,title,region,agenda,context,summary,total,evidence
0,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_1,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의석을 정돈하여 주시기 ...,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...
1,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_2,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의사팀장 수고하셨습니다....,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...
2,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_3,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제2항 ...,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...
3,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_4,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제3항 ...,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...
4,2001,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,AGENDA_1,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록 완주 의석을 정돈...,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...


In [84]:
train.shape

(2994, 8)

In [85]:
train.columns

Index(['id', 'title', 'region', 'agenda', 'context', 'summary', 'total',
       'evidence'],
      dtype='object')

In [83]:
train.to_csv('context_evidence.csv')